# Destiny 2 ML practice project

Many thanks to Garrick Fernandez (https://github.com/garrickf) for code outline for Bungie API data collection!

This will be an exploration of various data science concepts, with no set plan as of now other than to explore the data available. A couple ideas could be: <br>
-Identifying hackers in trails  <br>
-Predicting which team will win in PvP  <br>
-Identify trends which increase player engagement eg. good trials weapon - hypothesis testing??<br>

In [186]:
HEADERS = {"X-API-Key":os.environ["BUNGIE_API_KEY"]}
weapon = requests.get("https://www.bungie.net/platform/Destiny/Manifest/InventoryItem/1274330687/", headers=HEADERS)
inventoryItem = weapon.json()
print(inventoryItem['Response']['data']['inventoryItem']['itemName'])

Gjallarhorn


In [188]:
game = requests.get("https://www.bungie.net/platform/Destiny2/Stats/PostGameCarnageReport/84005478/", headers=HEADERS)
report = game.json()
report

{'Response': {'period': '2017-09-08T04:23:17Z',
  'startingPhaseIndex': 0,
  'activityWasStartedFromBeginning': False,
  'activityDetails': {'referenceId': 2266954523,
   'directorActivityHash': 2266954523,
   'instanceId': '84005478',
   'mode': 2,
   'modes': [7, 2],
   'isPrivate': False,
   'membershipType': 0},
  'entries': [{'standing': 0,
    'score': {'basic': {'value': 0.0, 'displayValue': '0'}},
    'player': {'destinyUserInfo': {'iconPath': '/common/destiny2_content/icons/ecd4608be3fb1c02755a96dc104dc003.jpg',
      'crossSaveOverride': 2,
      'applicableMembershipTypes': [3, 2],
      'isPublic': True,
      'membershipType': 2,
      'membershipId': '4611686018450981322',
      'displayName': 'rockerman91',
      'bungieGlobalDisplayName': 'rockerman91',
      'bungieGlobalDisplayNameCode': 4972},
     'characterClass': 'Titan',
     'classHash': 3655393761,
     'raceHash': 3887404748,
     'genderHash': 3111576190,
     'characterLevel': 20,
     'lightLevel': 283,
   

In [259]:
try1 = requests.get("https://www.bungie.net/Platform/User/GetBungieNetUserById/1/",headers=HEADERS)
print(try1)
#try1.json()

<Response [200]>


In [260]:
try2 = requests.get("https://www.bungie.net/Platform/Trending/Categories/", headers=HEADERS)
#try2.json()

In [261]:
try3 = requests.get("https://www.bungie.net/Platform//HelloWorld/", headers=HEADERS)
#try3.json()

In [262]:
try4 = requests.get("https://www.bungie.net/Platform//GlobalAlerts/", headers=HEADERS)
#try4.json()

In [263]:
try5 = requests.get("https://bungie.net/Platform/Destiny2/Stats/PostGameCarnageReport/" + str(7234123546),headers=HEADERS)
#try5.json()

In [264]:
try6 = requests.get("https://bungie.net/Platform/Destiny2/SearchDestinyPlayer/All/" + 'newksie', headers=HEADERS)
#try6.json()

In [265]:
try7 = requests.get("https://www.bungie.net/Platform/User/Search/Prefix/newksie/0/", headers=HEADERS)
mymembershipId = try7.json()['Response']['searchResults'][0]['destinyMemberships'][0]['membershipId']
mymembershipId
#try8 = requests.get("https://www.bungie.net/Platform/Destiny2/3/Profile/"+str(mymembershipId)+'/',headers=HEADERS)
#try8.json()

'4611686018467732204'

In [266]:
import json
import os
import time
from urllib.parse import urljoin
import pandas as pd
import requests

def get_activity_period(activity):
    return activity["period"]


def get_activity_reference_id(activity):
    return activity["activityDetails"]["referenceId"]


def get_activity_instance_id(activity):
    # Used for looking up post-game carnage reports (PGCRs)
    return activity["activityDetails"]["instanceId"]


def get_activity_values(activity):
    return activity["values"]


def get_value_kills(values):
    if "kills" in values:
        return values["kills"]["basic"]["value"]
    else:
        return None


def get_value_deaths(values):
    if "deaths" in values:
        return values["deaths"]["basic"]["value"]
    else:
        return None


def get_value_assists(values):
    if "assists" in values:
        return values["assists"]["basic"]["value"]
    else:
        return None


def get_value_score(values):
    if "score" in values:
        return values["score"]["basic"]["value"]
    else:
        return None


def get_value_completed(values):
    # 1 if completed
    if "completed" in values:
        return values["completed"]["basic"]["value"]
    else:
        return None


def get_value_opponents_defeated(values):
    if "opponentsDefeated" in values:
        return values["opponentsDefeated"]["basic"]["value"]
    else:
        return None


def get_value_combat_efficiency(values):
    if "efficiency" in values:
        return values["efficiency"]["basic"]["value"]
    else:
        return None


def get_value_kdr(values):
    if "killsDeathsRatio" in values:
        return values["killsDeathsRatio"]["basic"]["value"]
    else:
        return None


def get_value_kda(values):
    if "killsDeathsAssists" in values:
        return values["killsDeathsAssists"]["basic"]["value"]
    else:
        return None


def get_value_activity_duration(values, in_units_of="seconds"):
    if "activityDurationSeconds" in values:
        return values["activityDurationSeconds"]["basic"]["value"]
    else:
        return None


def get_value_team(values):
    if "team" in values:
        return values["team"]["basic"]["value"]
    else:
        return None


def get_value_standing(values):
    # The API marks 1 as defeat, 0 as victory, so 1-***
    if "standing" in values:
        return 1 - values["standing"]["basic"]["value"]
    else: 
        return None


def get_value_team_score(values):
    if "teamScore" in values:
        return values["teamScore"]["basic"]["value"]
    else:
        return None


def get_activity_history(character_data, params=None):
    path = urljoin(API_URL, GET_ACTIVITY_HISTORY.format(**character_data))
    r = requests.get(path, headers=HEADERS, params=params)

    activities = r.json()["Response"]["activities"]
    print(f"{len(activities)} activities found")

    history = []

    for activity in activities:
        period = get_activity_period(activity)
        details = activity["activityDetails"]
        activity_name = get_director_activity_name(activity)
        values = get_activity_values(activity)
        id = get_activity_instance_id(activity)
        print(f"{period}, {activity_name}, {id}")
        history.append(
            {
                "period": period,
                "activity_name": activity_name,
                "details": details,
                "values": values,
            }
        )

    return history

In [267]:
STARTING_ACTIVITY_ID = 8400554258
ENDING_ACTIVITY_ID = STARTING_ACTIVITY_ID + int(10)

API_URL = "https://www.bungie.net/Platform/"
GET_ACTIVITY_HISTORY = "Destiny2/{membershipType}/Account/{destinyMembershipId}/Character/{characterId}/Stats/Activities/"
GET_POST_GAME_CARNAGE_REPORT = "Destiny2/Stats/PostGameCarnageReport/{activityId}/"

HEADERS = {"X-API-Key":os.environ["BUNGIE_API_KEY"]}

In [268]:
instance_id=STARTING_ACTIVITY_ID+2000000
path = urljoin(API_URL, GET_POST_GAME_CARNAGE_REPORT.format(**{"activityId": instance_id}))
r = requests.get(path, headers=HEADERS, timeout=1)
pgcr = r.json()["Response"]
pgcr

{'period': '2021-05-08T01:39:17Z',
 'startingPhaseIndex': 0,
 'activityWasStartedFromBeginning': False,
 'activityDetails': {'referenceId': 1517708944,
  'directorActivityHash': 1517708944,
  'instanceId': '8402554258',
  'mode': 2,
  'modes': [7, 2],
  'isPrivate': False,
  'membershipType': 0},
 'entries': [{'standing': 0,
   'score': {'basic': {'value': 0.0, 'displayValue': '0'}},
   'player': {'destinyUserInfo': {'iconPath': '/common/destiny2_content/icons/a8170d3363513067f140c17ea0c4c96e.jpg',
     'crossSaveOverride': 0,
     'applicableMembershipTypes': [1],
     'isPublic': True,
     'membershipType': 1,
     'membershipId': '4611686018493721496',
     'displayName': 'NDL Prodigy5814',
     'bungieGlobalDisplayName': 'NDL Prodigy5814',
     'bungieGlobalDisplayNameCode': 1077},
    'characterClass': 'Hunter',
    'classHash': 671679327,
    'raceHash': 3887404748,
    'genderHash': 3111576190,
    'characterLevel': 50,
    'lightLevel': 1320,
    'emblemHash': 1230660648},
   

In [269]:
#pgcr["activityDetails"]["mode"]
#pgcr

In [270]:
# LOOK HERE
# Scrapes API for activities of a given mode, specified by 
# https://bungie-net.github.io/multi/
# schema_Destiny-HistoricalStats-Definitions-DestinyActivityModeType.html#schema_Destiny-
# HistoricalStats-Definitions-DestinyActivityModeType

def single_scrape(activity_id,mode):
    path = urljoin(API_URL, GET_POST_GAME_CARNAGE_REPORT.format(**{"activityId": activity_id}))
    r = requests.get(path, headers=HEADERS)
    #r = requests.get(path, headers=HEADERS, timeout=1)
    pgcr = r.json()["Response"]
    
    period = get_activity_period(pgcr)
    director_activity_mode = pgcr["activityDetails"]["mode"]
    if director_activity_mode!=mode:
        return None

    player_entries = {}
    for i, entry in enumerate(entries):
        characterId = entry["characterId"]
        player_entries[f"player{i + 1}_char_id"] = characterId

        # Values (kills, assists, etc.)
        if "values" not in entry: 
            continue
        
        values = entry["values"]
        player_entries[f"player{i + 1}_kills"] = get_value_kills(values)
        player_entries[f"player{i + 1}_deaths"] = get_value_deaths(values)
        player_entries[f"player{i + 1}_assists"] = get_value_assists(values)
        player_entries[f"player{i + 1}_kdr"] = get_value_kdr(values)
        player_entries[f"player{i + 1}_kda"] = get_value_kda(values)
        player_entries[f"player{i + 1}_efficiency"] = get_value_combat_efficiency(values)
        player_entries[f"player{i + 1}_score"] = get_value_score(values)
        player_entries[f"player{i + 1}_standing"] = get_value_standing(values)
        player_entries[f"player{i + 1}_team_score"] = get_value_team_score(values)
        player_entries[f"player{i + 1}_activity_duration"] = get_value_activity_duration(values)

    # Return dict
    return {
        "instance_id": activity_id,
        "period": period,
        "director_activity_mode": director_activity_mode,
        **player_entries,
    }

In [271]:
#LOOK HERE
#Loops single scrape for peroid described
def multi_scrape(start, end, mode):
    dataset=[]
    for activity_id in range(start,end):
        data_point = single_scrape(activity_id,mode)
        if data_point != None:
            dataset.append(data_point)
    df = pd.DataFrame(dataset)
    return df
    #df.to_csv("test.csv")

In [272]:
week1start=5866000000
week1end=5900000000
week1start+100000
#print(week1end-week1start)
#multi_scrape(5866000000,590000000, 84)

5866100000

In [273]:
#Trying to find id where trials of osiris begins, ie, 13th March 2020
trialsgame = requests.get("https://www.bungie.net/platform/Destiny2/Stats/PostGameCarnageReport/5876000000/", headers=HEADERS)
trialsreport = trialsgame.json()
trialsreport


{'Response': {'period': '2020-03-14T10:57:39Z',
  'startingPhaseIndex': -1,
  'activityWasStartedFromBeginning': False,
  'activityDetails': {'referenceId': 3966792859,
   'directorActivityHash': 3966792859,
   'instanceId': '5876000000',
   'mode': 6,
   'modes': [7, 6],
   'isPrivate': False,
   'membershipType': 3},
  'entries': [{'standing': 0,
    'score': {'basic': {'value': 0.0, 'displayValue': '0'}},
    'player': {'destinyUserInfo': {'iconPath': '/common/destiny2_content/icons/cb75fe0f22015ba50953ec3a0700d07a.jpg',
      'crossSaveOverride': 0,
      'applicableMembershipTypes': [3],
      'isPublic': True,
      'membershipType': 3,
      'membershipId': '4611686018485916336',
      'displayName': 'RUSSIAN БЫДЛОS',
      'bungieGlobalDisplayName': ''},
     'characterClass': 'Titan',
     'classHash': 3655393761,
     'raceHash': 3887404748,
     'genderHash': 3111576190,
     'characterLevel': 50,
     'lightLevel': 942,
     'emblemHash': 185321779},
    'characterId': '230

In [274]:
multi_scrape(5876000000,5876000000+10,84)

""
